# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [2]:
data = pd.read_csv('customer_product_sales.csv')

In [3]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [4]:
cantidad=data.groupby(['CustomerID','ProductName'],as_index=False)['Quantity'].sum()
cantidad.head()

,CustomerID,ProductName,Quantity
0,33,Apricots - Dried,1
1,33,Assorted Desserts,1
2,33,Bandage - Flexible Neon,1
3,33,"Bar Mix - Pina Colada, 355 Ml",1
4,33,"Beans - Kidney, Canned",1


## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [5]:
matrix=pd.pivot_table(cantidad,index='ProductName',columns='CustomerID',values='Quantity').fillna(0)
matrix.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Appetizer - Mini Egg Roll, Shrimp",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Appetizer - Mushroom Tart,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0
Appetizer - Sausage Rolls,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,25.0,25.0,25.0,0.0,25.0,0.0
Apricots - Dried,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [6]:
matrixt=matrix.T
matrixt

ProductName,Anchovy Paste - 56 G Tube,"Appetizer - Mini Egg Roll, Shrimp",Appetizer - Mushroom Tart,Appetizer - Sausage Rolls,Apricots - Dried,Apricots - Halves,Apricots Fresh,Arizona - Green Tea,Artichokes - Jerusalem,Assorted Desserts,...,"Wine - White, Colubia Cresh","Wine - White, Mosel Gold","Wine - White, Schroder And Schyl",Wine - Wyndham Estate Bin 777,Wonton Wrappers,Yeast Dry - Fermipan,Yoghurt Tubes,"Yogurt - Blueberry, 175 Gr",Yogurt - French Vanilla,Zucchini - Yellow
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
200,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
264,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
356,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
412,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,0.0,0.0,0.0,25.0,0.0,50.0,0.0,25.0,0.0,0.0,...,0.0,25.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
98069,0.0,0.0,0.0,25.0,0.0,25.0,0.0,0.0,0.0,25.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
98159,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,50.0,0.0,0.0,0.0,0.0,25.0,0.0,0.0,0.0


In [7]:
squareform(pdist(matrixt, metric='euclidean'))

array([[  0.        ,  11.91637529,  10.48808848, ..., 228.62851966,
        239.        , 229.77380181],
       [ 11.91637529,   0.        ,  11.74734012, ..., 228.01096465,
        239.03765394, 229.70415756],
       [ 10.48808848,  11.74734012,   0.        , ..., 228.08112592,
        238.26665734, 229.77380181],
       ...,
       [228.62851966, 228.01096465, 228.08112592, ...,   0.        ,
        304.13812651, 305.16389039],
       [239.        , 239.03765394, 238.26665734, ..., 304.13812651,
          0.        , 303.10889132],
       [229.77380181, 229.70415756, 229.77380181, ..., 305.16389039,
        303.10889132,   0.        ]])

In [8]:
1+squareform(pdist(matrixt, metric='euclidean'))

array([[  1.        ,  12.91637529,  11.48808848, ..., 229.62851966,
        240.        , 230.77380181],
       [ 12.91637529,   1.        ,  12.74734012, ..., 229.01096465,
        240.03765394, 230.70415756],
       [ 11.48808848,  12.74734012,   1.        , ..., 229.08112592,
        239.26665734, 230.77380181],
       ...,
       [229.62851966, 229.01096465, 229.08112592, ...,   1.        ,
        305.13812651, 306.16389039],
       [240.        , 240.03765394, 239.26665734, ..., 305.13812651,
          1.        , 304.10889132],
       [230.77380181, 230.70415756, 230.77380181, ..., 306.16389039,
        304.10889132,   1.        ]])

In [9]:
1/(1+squareform(pdist(matrixt, metric='euclidean')))

array([[1.        , 0.0774211 , 0.08704668, ..., 0.00435486, 0.00416667,
        0.00433325],
       [0.0774211 , 1.        , 0.07844774, ..., 0.0043666 , 0.00416601,
        0.00433456],
       [0.08704668, 0.07844774, 1.        , ..., 0.00436527, 0.00417944,
        0.00433325],
       ...,
       [0.00435486, 0.0043666 , 0.00436527, ..., 1.        , 0.0032772 ,
        0.00326622],
       [0.00416667, 0.00416601, 0.00417944, ..., 0.0032772 , 1.        ,
        0.0032883 ],
       [0.00433325, 0.00433456, 0.00433325, ..., 0.00326622, 0.0032883 ,
        1.        ]])

In [10]:
dist_eucl=1/(1+squareform(pdist(matrixt, metric='euclidean')))
distance = pd.DataFrame(dist_eucl, index=matrixt.index, columns=matrixt.index)
distance

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.077421,0.087047,0.081800,0.080634,0.082709,0.074573,0.083020,0.081503,0.080070,...,0.004811,0.004669,0.004412,0.005019,0.004312,0.004515,0.004583,0.004355,0.004167,0.004333
200,0.077421,1.000000,0.078448,0.076435,0.073693,0.075255,0.075956,0.076435,0.077674,0.076923,...,0.004824,0.004681,0.004431,0.005047,0.004311,0.004521,0.004614,0.004367,0.004166,0.004335
264,0.087047,0.078448,1.000000,0.080070,0.081800,0.080350,0.076923,0.080634,0.082100,0.078448,...,0.004822,0.004674,0.004416,0.005035,0.004322,0.004543,0.004595,0.004365,0.004179,0.004333
356,0.081800,0.076435,0.080070,1.000000,0.076435,0.078187,0.075025,0.082403,0.077171,0.075956,...,0.004816,0.004671,0.004416,0.005038,0.004310,0.004526,0.004578,0.004365,0.004175,0.004339
412,0.080634,0.073693,0.081800,0.076435,1.000000,0.078711,0.075025,0.082403,0.078187,0.078448,...,0.004810,0.004702,0.004414,0.005034,0.004318,0.004530,0.004578,0.004367,0.004177,0.004349
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,0.004515,0.004521,0.004543,0.004526,0.004530,0.004533,0.004534,0.004526,0.004528,0.004523,...,0.003509,0.003509,0.003300,0.003653,0.003579,1.000000,0.003638,0.003418,0.003255,0.003245
98069,0.004583,0.004614,0.004595,0.004578,0.004578,0.004578,0.004589,0.004581,0.004585,0.004592,...,0.003509,0.003509,0.003369,0.003509,0.003496,0.003638,1.000000,0.003469,0.003345,0.003381
98159,0.004355,0.004367,0.004365,0.004365,0.004367,0.004363,0.004366,0.004363,0.004369,0.004365,...,0.003509,0.003565,0.003322,0.003431,0.003322,0.003418,0.003469,1.000000,0.003277,0.003266


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [11]:
similarities_33 = distance[33].sort_values(ascending = False)[1:6]
similarities_33

CustomerID
264     0.087047
3535    0.087047
3317    0.087047
2503    0.085983
3305    0.085638
Name: 33, dtype: float64

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [12]:
cantidad_33=cantidad[cantidad.CustomerID.isin(similarities_33.index)]
cantidad_33.head()

,CustomerID,ProductName,Quantity
131,264,Apricots - Halves,1
132,264,Apricots Fresh,1
133,264,Bacardi Breezer - Tropical,1
134,264,Bagel - Plain,1
135,264,Banana - Leaves,1


## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [13]:
productos_33=cantidad_33.groupby('ProductName',as_index=False).Quantity.sum().sort_values('Quantity',ascending=False)
productos_33.head()

,ProductName,Quantity
33,Butter - Unsalted,3
203,Wine - Ej Gallo Sierra Valley,3
186,Towels - Paper / Kraft,3
168,Soup - Campbells Bean Medley,3
199,Wine - Blue Nun Qualitatswein,3


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [14]:
merge=matrix.merge(productos_33,on='ProductName')
merge

,ProductName,33,200,264,356,412,464,477,639,649,...,97753,97769,97793,97900,97928,98069,98159,98185,98200,Quantity
0,Anchovy Paste - 56 G Tube,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1,Appetizer - Mushroom Tart,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0,1
2,Appetizer - Sausage Rolls,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,25.0,25.0,25.0,0.0,25.0,0.0,1
3,Apricots - Dried,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
4,Apricots - Halves,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,25.0,50.0,25.0,0.0,25.0,25.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
213,"Wine - White, Schroder And Schyl",0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,25.0,0.0,25.0,0.0,0.0,0.0,0.0,1
214,Wine - Wyndham Estate Bin 777,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0,2
215,Wonton Wrappers,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
216,Yoghurt Tubes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0,0.0,2


In [15]:
merge[merge[33]==0]

,ProductName,33,200,264,356,412,464,477,639,649,...,97753,97769,97793,97900,97928,98069,98159,98185,98200,Quantity
0,Anchovy Paste - 56 G Tube,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1,Appetizer - Mushroom Tart,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0,1
2,Appetizer - Sausage Rolls,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,25.0,25.0,25.0,0.0,25.0,0.0,1
4,Apricots - Halves,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,25.0,50.0,25.0,0.0,25.0,25.0,1
5,Apricots Fresh,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
212,Wine - White Cab Sauv.on,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
213,"Wine - White, Schroder And Schyl",0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,25.0,0.0,25.0,0.0,0.0,0.0,0.0,1
214,Wine - Wyndham Estate Bin 777,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0,2
215,Wonton Wrappers,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


In [16]:
merge[merge[33]==0].sort_values('Quantity',ascending=False).head()

,ProductName,33,200,264,356,412,464,477,639,649,...,97753,97769,97793,97900,97928,98069,98159,98185,98200,Quantity
199,Wine - Blue Nun Qualitatswein,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0,3
168,Soup - Campbells Bean Medley,0.0,0.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0,...,0.0,0.0,25.0,50.0,25.0,0.0,0.0,0.0,0.0,3
203,Wine - Ej Gallo Sierra Valley,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,25.0,50.0,0.0,25.0,0.0,0.0,0.0,0.0,3
33,Butter - Unsalted,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0,3
216,Yoghurt Tubes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0,0.0,2


In [17]:
top5=merge.ProductName[:5].values
top5

array(['Anchovy Paste - 56 G Tube', 'Appetizer - Mushroom Tart',
       'Appetizer - Sausage Rolls', 'Apricots - Dried',
       'Apricots - Halves'], dtype=object)

## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [18]:
recomendations={}
customers=data.CustomerID.unique()
for x in customers:
    similarities_x = distance[x].sort_values(ascending = False)[1:6]
    cantidad_x=cantidad[cantidad.CustomerID.isin(similarities_x.index)]
    productos_x=cantidad_x.groupby('ProductName',as_index=False).Quantity.sum().sort_values('Quantity',ascending=False)
    merge_x=matrix.merge(productos_x,on='ProductName')
    top5_x=merge_x.ProductName[:5].values
    recomendations[x]=top5_x
    

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [19]:
recomendaciones=pd.DataFrame.from_dict(recomendations,orient='index',columns=['Recomendación 1','Recomendación 2','Recomendación 3','Recomendación 4','Recomendación 5']).reset_index()
recomendaciones.head()

,index,Recomendación 1,Recomendación 2,Recomendación 3,Recomendación 4,Recomendación 5
0,61288,Appetizer - Mushroom Tart,Apricots - Halves,Arizona - Green Tea,Assorted Desserts,Bacardi Breezer - Tropical
1,77352,Anchovy Paste - 56 G Tube,"Appetizer - Mini Egg Roll, Shrimp",Arizona - Green Tea,Assorted Desserts,Bacardi Breezer - Tropical
2,40094,Anchovy Paste - 56 G Tube,Appetizer - Sausage Rolls,Apricots Fresh,Arizona - Green Tea,Artichokes - Jerusalem
3,23548,Anchovy Paste - 56 G Tube,"Appetizer - Mini Egg Roll, Shrimp",Appetizer - Mushroom Tart,Appetizer - Sausage Rolls,Apricots - Dried
4,78981,"Appetizer - Mini Egg Roll, Shrimp",Appetizer - Mushroom Tart,Apricots - Halves,Apricots Fresh,Artichokes - Jerusalem


In [20]:
recomendaciones.rename(columns = {'index':'ID cliente'}, inplace = True)
recomendaciones.head()

,ID cliente,Recomendación 1,Recomendación 2,Recomendación 3,Recomendación 4,Recomendación 5
0,61288,Appetizer - Mushroom Tart,Apricots - Halves,Arizona - Green Tea,Assorted Desserts,Bacardi Breezer - Tropical
1,77352,Anchovy Paste - 56 G Tube,"Appetizer - Mini Egg Roll, Shrimp",Arizona - Green Tea,Assorted Desserts,Bacardi Breezer - Tropical
2,40094,Anchovy Paste - 56 G Tube,Appetizer - Sausage Rolls,Apricots Fresh,Arizona - Green Tea,Artichokes - Jerusalem
3,23548,Anchovy Paste - 56 G Tube,"Appetizer - Mini Egg Roll, Shrimp",Appetizer - Mushroom Tart,Appetizer - Sausage Rolls,Apricots - Dried
4,78981,"Appetizer - Mini Egg Roll, Shrimp",Appetizer - Mushroom Tart,Apricots - Halves,Apricots Fresh,Artichokes - Jerusalem


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [21]:
squareform(pdist(matrixt, metric='jaccard'))

array([[0.        , 0.94308943, 0.88785047, ..., 1.        , 1.        ,
        1.        ],
       [0.94308943, 0.        , 0.92741935, ..., 1.        , 1.        ,
        1.        ],
       [0.88785047, 0.92741935, 0.        , ..., 1.        , 1.        ,
        1.        ],
       ...,
       [1.        , 1.        , 1.        , ..., 0.        , 0.90551181,
        0.95081967],
       [1.        , 1.        , 1.        , ..., 0.90551181, 0.        ,
        0.9       ],
       [1.        , 1.        , 1.        , ..., 0.95081967, 0.9       ,
        0.        ]])

In [22]:
1+squareform(pdist(matrixt, metric='jaccard'))

array([[1.        , 1.94308943, 1.88785047, ..., 2.        , 2.        ,
        2.        ],
       [1.94308943, 1.        , 1.92741935, ..., 2.        , 2.        ,
        2.        ],
       [1.88785047, 1.92741935, 1.        , ..., 2.        , 2.        ,
        2.        ],
       ...,
       [2.        , 2.        , 2.        , ..., 1.        , 1.90551181,
        1.95081967],
       [2.        , 2.        , 2.        , ..., 1.90551181, 1.        ,
        1.9       ],
       [2.        , 2.        , 2.        , ..., 1.95081967, 1.9       ,
        1.        ]])

In [23]:
1/(1+squareform(pdist(matrixt, metric='jaccard')))

array([[1.        , 0.51464435, 0.52970297, ..., 0.5       , 0.5       ,
        0.5       ],
       [0.51464435, 1.        , 0.51882845, ..., 0.5       , 0.5       ,
        0.5       ],
       [0.52970297, 0.51882845, 1.        , ..., 0.5       , 0.5       ,
        0.5       ],
       ...,
       [0.5       , 0.5       , 0.5       , ..., 1.        , 0.52479339,
        0.51260504],
       [0.5       , 0.5       , 0.5       , ..., 0.52479339, 1.        ,
        0.52631579],
       [0.5       , 0.5       , 0.5       , ..., 0.51260504, 0.52631579,
        1.        ]])

In [24]:
dist_jac=1/(1+squareform(pdist(matrixt, metric='jaccard')))
distance = pd.DataFrame(dist_jac, index=matrixt.index, columns=matrixt.index)
distance

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.514644,0.529703,0.524038,0.513043,0.516129,0.510549,0.509174,0.511211,0.513889,...,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000
200,0.514644,1.000000,0.518828,0.514523,0.503704,0.512000,0.520325,0.510204,0.518828,0.519313,...,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000
264,0.529703,0.518828,1.000000,0.515695,0.527027,0.515419,0.510460,0.508772,0.515419,0.511013,...,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000
356,0.524038,0.514523,0.515695,1.000000,0.517094,0.510917,0.508403,0.521531,0.510823,0.513393,...,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000
412,0.513043,0.503704,0.527027,0.517094,1.000000,0.512605,0.516529,0.520362,0.510373,0.515284,...,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,...,0.518987,0.525581,0.515021,0.521951,0.520362,1.000000,0.524887,0.525346,0.511628,0.504000
98069,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,...,0.516129,0.512821,0.514644,0.511013,0.526549,0.524887,1.000000,0.527027,0.517510,0.523404
98159,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,...,0.512500,0.525822,0.520000,0.513889,0.515152,0.525346,0.527027,1.000000,0.524793,0.512605


In [25]:
recomendations={}
customers=data.CustomerID.unique()
for x in customers:
    similarities_x = distance[x].sort_values(ascending = False)[1:6]
    cantidad_x=cantidad[cantidad.CustomerID.isin(similarities_x.index)]
    productos_x=cantidad_x.groupby('ProductName',as_index=False).Quantity.sum().sort_values('Quantity',ascending=False)
    merge_x=matrix.merge(productos_x,on='ProductName')
    top5_x=merge_x.ProductName[:5].values
    recomendations[x]=top5_x

In [26]:
recomendaciones=pd.DataFrame.from_dict(recomendations,orient='index',columns=['Recomendación 1','Recomendación 2','Recomendación 3','Recomendación 4','Recomendación 5']).reset_index()
recomendaciones.head()

,index,Recomendación 1,Recomendación 2,Recomendación 3,Recomendación 4,Recomendación 5
0,61288,Anchovy Paste - 56 G Tube,Appetizer - Sausage Rolls,Apricots Fresh,Assorted Desserts,Baking Powder
1,77352,Anchovy Paste - 56 G Tube,"Appetizer - Mini Egg Roll, Shrimp",Appetizer - Mushroom Tart,Apricots - Halves,Apricots Fresh
2,40094,Anchovy Paste - 56 G Tube,"Appetizer - Mini Egg Roll, Shrimp",Appetizer - Mushroom Tart,Appetizer - Sausage Rolls,Apricots - Halves
3,23548,Anchovy Paste - 56 G Tube,Appetizer - Mushroom Tart,Apricots - Dried,Apricots - Halves,Arizona - Green Tea
4,78981,Anchovy Paste - 56 G Tube,Appetizer - Sausage Rolls,Apricots - Halves,Apricots Fresh,Artichokes - Jerusalem


In [27]:
recomendaciones.rename(columns = {'index':'ID cliente'}, inplace = True)
recomendaciones.head()

,ID cliente,Recomendación 1,Recomendación 2,Recomendación 3,Recomendación 4,Recomendación 5
0,61288,Anchovy Paste - 56 G Tube,Appetizer - Sausage Rolls,Apricots Fresh,Assorted Desserts,Baking Powder
1,77352,Anchovy Paste - 56 G Tube,"Appetizer - Mini Egg Roll, Shrimp",Appetizer - Mushroom Tart,Apricots - Halves,Apricots Fresh
2,40094,Anchovy Paste - 56 G Tube,"Appetizer - Mini Egg Roll, Shrimp",Appetizer - Mushroom Tart,Appetizer - Sausage Rolls,Apricots - Halves
3,23548,Anchovy Paste - 56 G Tube,Appetizer - Mushroom Tart,Apricots - Dried,Apricots - Halves,Arizona - Green Tea
4,78981,Anchovy Paste - 56 G Tube,Appetizer - Sausage Rolls,Apricots - Halves,Apricots Fresh,Artichokes - Jerusalem


In [28]:
#Las recomendaciones sí cambiaron respecto a las que se habían dado antes cuando la métrica de distancia fue Euclideana para los 5 clientes que se muestran. 